<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Машинное обучение и майнинг данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# Занятие 4. Продвинутые методы классификации и регрессии. Переобучение

## Практика. Случайный лес в соревновании Kaggle Inclass по автострахованию

<a href="https://inclass.kaggle.com/c/hse-addprofeduc-ml-contest">Соревнование</a>, исходное <a href="http://microsoftbi.ru/2015/06/06/hackathon2015ml/">описание</a> задачи. 

Задача бинарной классификации. Имеются автомобили, для которых указан регистрационный номер и марка, и выплаты страховой компании по инцидентам с участием данного автомобиля. Страховая компания для себя решает, много она заплатила или мало. 

Объекты - автомобили.

Признаки:

- Регистрационный номер автомобиля (auto_number, уникальный, строка)
- Марка автомобиля (auto_brand, строка)
- Тип выплаты (too_much) (много/мало, 1 или 0)
- Сумма выплаты при попадании водителя в аварию (compensated, целое положительное число)

### Загрузка и первичный анализ данных

In [1]:
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
%pylab inline
figsize(12, 8)
import seaborn as sns
import pandas as pd
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import sys
sys.path.append('../../scripts/')
from load_car_insurance_with_region import load_train_and_test

Populating the interactive namespace from numpy and matplotlib


**Считаем обучающую и тестовую выборку, создав объекты Pandas DataFrame.**

In [2]:
train_df, y, test_df = load_train_and_test('../../data/car_insurance_train.csv',
                                          '../../data/car_insurance_test.csv')

In [3]:
train_df.head()

,auto_brand,compensated,region
id,,,
1,2,3200,21
2,5,6500,12
3,2,2100,9
4,2,2000,4
5,2,6100,21


**"Продвинутый" подход для работы с категориальными признаками - это считать статистики (например, max, min, median, квантили и т.д.) количественных признаков по категориальным. Для этого удобно использовать метод groupby pandas.DataFrame (при необходимости посмотрите документацию метода).**

In [4]:
#?train_df.groupby

**Пример. Считаем средние выплаты для каждой марки авто отдельно. Видно, что для разных марок средние выплаты могут существенно отличаться.**

In [8]:
mean_comp_by_brand = {auto_brand_id: sub_df['compensated'].mean() 
                      for (auto_brand_id, sub_df) 
                      in train_df.groupby("auto_brand")}

In [10]:
mean_comp_by_brand

{0.0: 4200.0,
 1.0: 3366.6666666666665,
 2.0: 7498.480243161094,
 3.0: 14312.5,
 4.0: 9988.235294117647,
 5.0: 7305.882352941177,
 6.0: 10672.222222222223,
 7.0: 5533.333333333333,
 8.0: 15162.5,
 9.0: 9442.857142857143,
 10.0: 5823.809523809524,
 11.0: 3100.0,
 12.0: 8500.0}

In [14]:
train_df['mean_by_brand'] = train_df['auto_brand'].apply(lambda x: 
                                                         mean_comp_by_brand[x])

In [15]:
train_df['mean_by_brand'].head()

id
1    7498.480243
2    7305.882353
3    7498.480243
4    7498.480243
5    7498.480243
Name: mean_by_brand, dtype: float64

**Создайте признаки "Медианные выплаты по маркам", "Средние выплаты по регионам" и т.п. (используйте функцию groupby). Посчитайте статистики mean, median, min и max (можно, конечно, и написать функцию, которая считает заданную статистику для заданного признака). Получится 8 новых признаков. После этого удалите исходные признаки auto_brand и region.**

In [ ]:
train_df['new_feature'] = # Ваш код здесь
test_df['new_feature'] = # Ваш код здесь

**Определите с помощью случайного леса важность имеющихся признаков. Здесь можно брать много (например, 1000) глубоких (например, глубины 5) деревьев решений. Используйте параметр random_state=42.**

In [ ]:
forest = # Ваш код здесь
# Ваш код здесь
pd.DataFrame(forest.feature_importances_,
             index=train_df.columns,
             columns=['Importance']).sort(['Importance'], ascending=False)

**Обучите на имеющейся выборке с 3 "лучшими" признаками случайный лес, переберите параметры глубины дерева от 1 до 4, а число деревьев - 100 или 500. Используйте 5-кратную кросс-валидацию и параметр random_state=42.**

In [ ]:
selected_features = # Ваш код здесь

tree_params = # Ваш код здесь

locally_best_clf = GridSearchCV # Ваш код здесь
locally_best_clf.fit(train_df[selected_features], y)

In [ ]:
locally_best_clf.best_params_

In [ ]:
locally_best_clf.best_score_

**Обучите на всей выборке случайный лес, с лучшими параметрами, определенными ранее. Используйте также параметр random_state=42.**

In [ ]:
final_model = # Ваш код здесь
final_model.fit(train_df[selected_features], y)

### Предсказание выплат для тестовой выборки

**Сделайте прогноз для объектов тестовой выборки.**

In [ ]:
predicted_labels # Ваш код здесь

**Запишите ответы в csv-файл и отправьте решение на Kaggle.**

In [ ]:
predicted_df = pd.DataFrame(predicted_labels,
                            index = np.arange(1, test_df.shape[0] + 1),
                            columns=["too_much"])
predicted_df.to_csv("../output/rf_with_region_tuned.csv", index_label="id")

## Ссылки:
- <a href="https://inclass.kaggle.com/c/hse-addprofeduc-ml-contest">Соревнование</a> на сайте Kaggle Inclass
- Исходное <a href="http://microsoftbi.ru/2015/06/06/hackathon2015ml/">описание</a> задачи
- [Документация](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) Scikit-learn по классу RandomForestClassifier